In [ ]:
# Registered data from CrossModa challenge by Mattias
# crossmoda_"target" -> crossmoda_"source"
#                 T2 -> T1
#             moving -> fixed

# Data originally stored in
# "/share/data_supergrover1/heinrich/crossmoda_convex/"
# 900 registrations (1F<-30M)x30

# "/share/data_supergrover1/heinrich/crossmoda_convex/"
# 900 registrations (1F<-30M)x30

# load with:
# contains nested dicts with dict[fixed_id][moving_id] with sparse label tensors.
# Use .to_dense() to unpack

# Script to reproduce

```python
load_set = "deeds"

if load_set == "convex":
    data_path = "/share/data_supergrover1/weihsbach/shared_data/tmp/tmp_convex/"
    convex_path =  "/share/data_supergrover1/heinrich/crossmoda_convex/"
    registered_files = glob.glob(data_path+"*.nii.gz")
    # registered_files = ["/share/data_supergrover1/weihsbach/shared_data/tmp/tmp_convex/Fcrossmoda108_M100.nii.gz"]
elif load_set == "deeds":
    data_path = "/share/data_supergrover1/heinrich/crossmoda_deeds/"
    convex_path =  "/share/data_supergrover1/heinrich/crossmoda_convex/"
    registered_files = glob.glob(data_path+"*seg.nii.gz")

dice_files_right = torch.load(convex_path+"dice_files_right.pth")
dice_files_left = torch.load(convex_path+"dice_files_left.pth")

orig_path = "/share/data_supergrover1/weihsbach/shared_data/tmp/CrossMoDa/L4_fine_localized_crop/"

# print(registered_files)
dice_files_right = torch.load(convex_path+"dice_files_right.pth")
dice_files_left = torch.load(convex_path+"dice_files_left.pth")

fixed_files_right = dice_files_right['target_tumour_right']
fixed_files_right = set([elem[0] for elem in fixed_files_right])
fixed_files_left = dice_files_left['target_tumour_left']
fixed_files_left = set([elem[0] for elem in fixed_files_left])

moving_files_right = dice_files_right['source_tumour_right']
moving_files_right = set([elem[0] for elem in moving_files_right])
moving_files_left = dice_files_left['source_tumour_left']
moving_files_left = set([elem[0] for elem in moving_files_left])

all_fixed_files = sorted(list(fixed_files_left.union(fixed_files_right)))
all_moving_files = sorted(list(moving_files_left.union(moving_files_right)))


def filter_ids(_file):
    num_id, lr_id = re.findall(r"(\d{1,3})_Label_([lr])", _file)[0]
    _id = f'{int(num_id):03d}{lr_id}'
    numeric_short = int(_id[:3])
    return (_id, numeric_short, _file)

fixed_ids = [filter_ids(f_name) for f_name in all_fixed_files]
moving_ids = [filter_ids(f_name) for f_name in all_moving_files]

data_dict = {}

used_fids_short = []
used_mids_short = []

def get_convex_fixed_moving_num(_file):
    fixed_num, moving_num = re.findall(r"(\d{1,3})r_M(\d{1,3})", _file)[0]
    fixed_num, moving_num = int(fixed_num), int(moving_num)
    return fixed_num, moving_num

def get_deeds_fixed_moving_num(_file):
    fixed_num, moving_num = re.findall(r"(\d{1,3})_L_M(\d{1,3})", _file)[0]
    fixed_num, moving_num = int(fixed_num), int(moving_num)
    return fixed_num, moving_num

for gz_file in registered_files:
    if load_set == "convex":
        fixed_num, moving_num = get_convex_fixed_moving_num(gz_file)
    elif load_set == "deeds":
        fixed_num, moving_num = get_deeds_fixed_moving_num(gz_file)
    used_fids_short.append(fixed_num)
    used_mids_short.append(moving_num)

used_fids_short = list(set(used_fids_short))
used_mids_short = list(set(used_mids_short))

fixed_ids = list(filter(lambda elem: elem[1] in used_fids_short, fixed_ids))
moving_ids = list(filter(lambda elem: elem[1] in used_mids_short, moving_ids))

print("fixed_ids", fixed_ids)
print("moving_ids", moving_ids)

orig_label_dict = {}

for _id, _, _file in fixed_ids:
    file_path = orig_path + "__omitted_labels_target_training__/" + _file
    file_path = file_path.replace("Label_l", "hrT2_l_Label")
    file_path = file_path.replace("Label_r", "hrT2_r_Label")
    if os.path.isfile(file_path):
        orig_label_dict[_id] = torch.tensor(nib.load(file_path).get_fdata())

warped_label_dict = {}
for gz_file in registered_files:
    if load_set == "convex":
        fixed_num, moving_num = get_convex_fixed_moving_num(gz_file)
        # gz_file = data_path + f"crossmoda_{fixed_num}_L{moving_num}.nii.gz"
    elif load_set == "deeds":
        fixed_num, moving_num = get_deeds_fixed_moving_num(gz_file)

    warped_label = torch.tensor(nib.load(gz_file).get_fdata()).cuda()

    fixed_id = list(filter(lambda elem: elem[1] == fixed_num, fixed_ids))
    if not fixed_id:
        continue
    else:
        fixed_id = fixed_id[0][0]
    moving_id = list(filter(lambda elem: elem[1] == moving_num, moving_ids))
    if not moving_id:
        continue
    else:
        moving_id = moving_id[0][0]

    dct = warped_label_dict.get(fixed_id, {})
    dct[moving_id] = warped_label.to_sparse()
    warped_label_dict[fixed_id] = dct

print(len(orig_label_dict))

for idx, gz_file in enumerate(registered_files):
    if load_set == "convex":
        fixed_num, moving_num = get_convex_fixed_moving_num(gz_file)
    elif load_set == "deeds":
        fixed_num, moving_num = get_deeds_fixed_moving_num(gz_file)

    fixed_res = list(filter(lambda elem: elem[1] == fixed_num, fixed_ids))
    if not fixed_res:
        continue
    else:
        fixed_id, _, fixed_file = fixed_res[0]

    moving_res = list(filter(lambda elem: elem[1] == moving_num, moving_ids))
    if not moving_res:
        continue
    else:
        moving_id, _, moving_file = moving_res[0]
    dct = data_dict.get(fixed_id, {})

    # orig_label = orig_label_dict[moving_id].cuda() # TODO
    orig_label = orig_label_dict[fixed_id].cuda()
    warped_label = warped_label_dict[fixed_id][moving_id].to_dense()

    dice = dice3d(F.one_hot(orig_label.long(), 3).unsqueeze(0),
          F.one_hot(warped_label.long(), 3).unsqueeze(0), one_hot_torch_style=True)

    dct[moving_id] = {
        'warped_label': warped_label.to_sparse(),
        'dice': dice
    }
    data_dict[fixed_id] = dct

    if idx % 60 == 0:
    # if len(orig_label.unique()) != len(warped_label.unique()) or len(orig_label.unique()) < 2:
        print(idx)
        print(orig_label.unique())
        print(warped_label.unique())
        print("Registred file: ", gz_file)
        print("Fixed file: ", fixed_file)
        print("Moving file: ", moving_file)
        print(f"fixed: {fixed_num}, moving: {moving_num}")
        print(f"fixed: {fixed_id}, moving: {moving_id}")
        print(dice)
        print()

torch.save(data_dict, THIS_SCRIPT_DIR+f"/data/crossmoda_{load_set}_registered.pth")
```